In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import torchvision.transforms as transforms


In [22]:
# Tạo dữ liệu giả lập (1000 mẫu, mỗi mẫu có 3 kênh 32x32, và 10 lớp)
num_samples = 1000
num_classes = 10
input_size = (3, 32, 32)  # Dữ liệu ảnh 3 kênh, 32x32

# Tạo dữ liệu và nhãn ngẫu nhiên
X = torch.randn(num_samples, *input_size)  # Dữ liệu ngẫu nhiên
y = torch.randint(0, num_classes, (num_samples,))  # Nhãn từ 0 đến 9

# Chia dữ liệu thành tập huấn luyện và kiểm thử (80/20)
train_size = int(0.8 * num_samples)
test_size = num_samples - train_size
train_data, test_data = random_split(TensorDataset(X, y), [train_size, test_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)


In [23]:
class CNNModel(nn.Module):
    def __init__(self, num_classes=10):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)  # (B, 16, 32, 32)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)  # (B, 32, 32, 32)
        self.pool = nn.MaxPool2d(2, 2)  # (B, 32, 16, 16)
        self.fc1 = nn.Linear(32 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [24]:
model = CNNModel(num_classes=10)
criterion = nn.CrossEntropyLoss()  # Dùng cho bài toán phân loại nhiều lớp
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [25]:
def train(model, train_loader, criterion, optimizer, num_epochs=10, device='cpu'):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            # Chuyển dữ liệu vào thiết bị (nếu có GPU)
            inputs, labels = inputs.to(device), labels.to(device)

            # Reset gradient
            optimizer.zero_grad()

            # Forward
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward và update weights
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


In [28]:
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    prediction = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            prediction.append(predicted)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy: {100 * correct / total:.2f}%')
    return prediction

# Kiểm tra xem có GPU không, nếu có dùng GPU để tăng tốc
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Huấn luyện và đánh giá
train(model, train_loader, criterion, optimizer, num_epochs=10, device=device)
evaluate(model, test_loader)


Epoch [1/10], Loss: 0.5912
Epoch [2/10], Loss: 0.3622
Epoch [3/10], Loss: 0.2062
Epoch [4/10], Loss: 0.1123
Epoch [5/10], Loss: 0.0734
Epoch [6/10], Loss: 0.0518
Epoch [7/10], Loss: 0.0378
Epoch [8/10], Loss: 0.0287
Epoch [9/10], Loss: 0.0234
Epoch [10/10], Loss: 0.0204
Accuracy: 10.00%


[tensor([5, 2, 8, 6, 0, 7, 6, 9, 7, 0, 5, 6, 6, 4, 2, 0, 5, 9, 5, 9, 5, 7, 0, 0,
         3, 0, 2, 4, 3, 3, 0, 4], device='cuda:0'),
 tensor([6, 7, 4, 9, 5, 6, 3, 0, 6, 0, 1, 3, 1, 0, 2, 1, 8, 4, 6, 1, 7, 4, 4, 3,
         7, 3, 5, 1, 1, 8, 8, 9], device='cuda:0'),
 tensor([2, 4, 0, 5, 0, 0, 3, 2, 6, 7, 0, 3, 6, 5, 2, 6, 1, 2, 1, 5, 7, 8, 6, 8,
         7, 2, 9, 4, 5, 5, 4, 1], device='cuda:0'),
 tensor([1, 1, 8, 7, 6, 0, 9, 2, 4, 3, 2, 2, 0, 6, 8, 8, 5, 9, 1, 9, 2, 9, 5, 4,
         7, 9, 2, 8, 6, 7, 1, 8], device='cuda:0'),
 tensor([7, 0, 8, 1, 8, 8, 4, 1, 7, 5, 9, 9, 2, 7, 9, 6, 5, 8, 5, 1, 0, 9, 1, 3,
         4, 8, 5, 4, 8, 8, 1, 3], device='cuda:0'),
 tensor([5, 0, 5, 6, 0, 5, 9, 4, 7, 6, 7, 7, 0, 7, 5, 3, 3, 2, 8, 6, 0, 5, 1, 9,
         8, 2, 6, 5, 0, 5, 0, 7], device='cuda:0'),
 tensor([5, 4, 6, 0, 0, 5, 9, 1], device='cuda:0')]

In [29]:
prediction = evaluate(model, test_loader)
prediction

Accuracy: 10.00%


[tensor([5, 2, 8, 6, 0, 7, 6, 9, 7, 0, 5, 6, 6, 4, 2, 0, 5, 9, 5, 9, 5, 7, 0, 0,
         3, 0, 2, 4, 3, 3, 0, 4], device='cuda:0'),
 tensor([6, 7, 4, 9, 5, 6, 3, 0, 6, 0, 1, 3, 1, 0, 2, 1, 8, 4, 6, 1, 7, 4, 4, 3,
         7, 3, 5, 1, 1, 8, 8, 9], device='cuda:0'),
 tensor([2, 4, 0, 5, 0, 0, 3, 2, 6, 7, 0, 3, 6, 5, 2, 6, 1, 2, 1, 5, 7, 8, 6, 8,
         7, 2, 9, 4, 5, 5, 4, 1], device='cuda:0'),
 tensor([1, 1, 8, 7, 6, 0, 9, 2, 4, 3, 2, 2, 0, 6, 8, 8, 5, 9, 1, 9, 2, 9, 5, 4,
         7, 9, 2, 8, 6, 7, 1, 8], device='cuda:0'),
 tensor([7, 0, 8, 1, 8, 8, 4, 1, 7, 5, 9, 9, 2, 7, 9, 6, 5, 8, 5, 1, 0, 9, 1, 3,
         4, 8, 5, 4, 8, 8, 1, 3], device='cuda:0'),
 tensor([5, 0, 5, 6, 0, 5, 9, 4, 7, 6, 7, 7, 0, 7, 5, 3, 3, 2, 8, 6, 0, 5, 1, 9,
         8, 2, 6, 5, 0, 5, 0, 7], device='cuda:0'),
 tensor([5, 4, 6, 0, 0, 5, 9, 1], device='cuda:0')]